#### 正文内容提取 & 内容符合度检测

In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import json
import csv
import requests
import base64

In [26]:
from difflib import SequenceMatcher

# 建立比较相似度方法
def similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [109]:
# 希望输出csv表格，因此首先创建文件

# 1. 创建文件对象
f = open('photo_content_check4.csv','a',newline='',encoding='utf-8')

# 2. 基于文件对象构建 csv写入对象
csv_writer = csv.writer(f)

# 3. 构建列表头
csv_writer.writerow(["图片名","正文内容","符合度"])

14

In [3]:
# 设定正文内容标准

stand = '有木有跟我一起免费参加大学生在线实训项目的,4大项目(抖音运营、竞品分析、阿里云认证等),利用课余时间锻炼自己的求职技能,丰富自己的个人简历。'

In [2]:
# 获取调用接口所需要的密钥

import requests 

# client_id 为官网获取的AK， client_secret 为官网获取的SK
host = 'https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials&client_id=kKRvSIhzMf5bqsAo82aQdcKI&client_secret=lqTolRXpNEBSaKUSgywK5mEotr1mvAWP'

# 获取response
response = requests.get(host)

# 显示api数据（密钥数据在内）
if response:
    print(response.json())

{'refresh_token': '25.0d04d1c58cf7c6a3f25ea72a4e2bd3c0.315360000.1916463166.282335-22592391', 'expires_in': 2592000, 'session_key': '9mzdDADonw9V1jFDS3fqa5769ORCdfNZvrGCu46tMlqt7uKfglg+WfAHJaQXWwZ1zlxwRyaPw902yD7KDmsuLAR28ptEWQ==', 'access_token': '24.7517952e71d8efd37df1633b0a8909aa.2592000.1603695166.282335-22592391', 'scope': 'public vis-ocr_ocr brain_ocr_scope brain_ocr_general brain_ocr_general_basic vis-ocr_business_license brain_ocr_webimage brain_all_scope brain_ocr_idcard brain_ocr_driving_license brain_ocr_vehicle_license vis-ocr_plate_number brain_solution brain_ocr_plate_number brain_ocr_accurate brain_ocr_accurate_basic brain_ocr_receipt brain_ocr_business_license brain_solution_iocr brain_qrcode brain_ocr_handwriting brain_ocr_passport brain_ocr_vat_invoice brain_numbers brain_ocr_business_card brain_ocr_train_ticket brain_ocr_taxi_receipt vis-ocr_household_register vis-ocr_vis-classify_birth_certificate vis-ocr_台湾通行证 vis-ocr_港澳通行证 vis-ocr_机动车购车发票识别 vis-ocr_机动车检验合格证识别 vis

In [5]:
# ocr文字识别 + 符合度检测

# 采用百度ocr接口，此处为接口请求地址前缀
request_url = "https://aip.baidubce.com/rest/2.0/ocr/v1/general_basic"

# 显示每张图的符合度情况
print('132张图片的正文的符合度：')

# 遍历132张图片
for i in range(1,2):
    try:
        
        #首先ocr获取全部内容信息
        
        # 打开文件
        f = open('./chushi/mmexport (%d).jpg'%i, 'rb')
        
        # 二进制方式打开图片文件
        img = base64.b64encode(f.read())
        
        # 记录文件二进制数据（调用参数）
        params = {"image":img}
        
        # 调用密钥（参数）
        access_token = '24.7517952e71d8efd37df1633b0a8909aa.2592000.1603695166.282335-22592391'
        
        # 完整请求地址
        request_url = request_url + "?access_token=" + access_token
        
        # 请求头（指定请求地址信息）
        headers = {'content-type': 'application/x-www-form-urlencoded'}
        
        # 得到ocr处理后的文本（response格式）
        response = requests.post(request_url, data=params, headers=headers)
        
        # 格式转化
        # 先转化为str
        text = response.text
        
        # 再转化为字典
        diat = json.loads(text)
        
        # 获取目标信息区域
        contont = diat['words_result']
        
        print(contont)
    except:
        pass

132张图片的正文的符合度：
[{'words': '中国移动图●'}, {'words': '中国移动'}, {'words': '2m>4@09:50'}, {'words': '详情'}, {'words': '·有木有跟我一起免费参加大学生在线'}, {'words': '实训项目的,4大项目(抖音运营'}, {'words': '竞品分析、阿里云认证等)①3,利用'}, {'words': '课余时间锻炼自己的求职技能,丰富'}, {'words': '自己的个人简历。'}, {'words': '在线免费报名参加在线实训项目,丰'}, {'words': '实习富个人简历,ge高薪ofer'}, {'words': '昨天20:52删除'}, {'words': '昨天20:52'}, {'words': '听参加过项目的同学说,这个项目对'}, {'words': '我们挺有帮助,文章最末端就可以报'}, {'words': '名参加。'}]


In [104]:
        # 符合度检测
        
        
        # 格式转化
        # 先转化为str
        text = response.text
        
        # 再转化为字典
        diat = json.loads(text)
        
        # 获取目标信息区域
        contont = diat['words_result']
        
        # 图片正文文本
        real = ''
              
        # 只获取正文内容
        
        
        # 通过遍历前10个信息段，找到正文位置
        for ii in range(10):
            
            # 因为百度ocr以行为单位识别，所以正文内容以行获取填充
            content = contont[ii]['words']
            
            # 通过寻找正文部分内容定位
            if('有木有'in content):
                
                # 通过正文部分的下一部分结束填充
                while('报名' not in content):
                    
                    # 将正文文本填充到字符串
                    real += content
                    
                    # 获取下一行
                    ii+=1
                    
                    # 获取文本
                    content = contont[ii]['words']
                    
                    
        # 将图片正文文本与标准文本进行对比，得到符合度
        result = similarity(real, stand)
        
        # 记录图片号、正文文本、符合度
        csv_writer.writerow([i,real,result])
        
        # 显示符合度情况
        print('图片',i,'：',result)
        
    except:
        pass

132张图片的正文的符合度：
图片 1 ： 0.9722222222222222
图片 2 ： 0.9861111111111112
图片 3 ： 0.9790209790209791
图片 4 ： 0.7407407407407407
图片 5 ： 0.9859154929577465
图片 6 ： 0.9722222222222222
图片 7 ： 0.9790209790209791
图片 8 ： 0.9790209790209791
图片 9 ： 0.9861111111111112
图片 10 ： 0.993006993006993
图片 11 ： 0.993006993006993
图片 12 ： 0.9790209790209791
图片 13 ： 0.9790209790209791
图片 14 ： 0.9861111111111112
图片 15 ： 0.993006993006993
图片 16 ： 0.9530201342281879
图片 17 ： 0.9790209790209791
图片 18 ： 0.9861111111111112
图片 19 ： 0.9859154929577465
图片 20 ： 0.9861111111111112
图片 21 ： 0.9861111111111112
图片 22 ： 0.9861111111111112
图片 23 ： 0.993006993006993
图片 24 ： 0.9790209790209791
图片 25 ： 0.9929078014184397
图片 26 ： 0.993006993006993
图片 27 ： 0.9861111111111112
图片 28 ： 0.9861111111111112
图片 29 ： 0.9861111111111112
图片 30 ： 0.9790209790209791
图片 31 ： 0.9859154929577465
图片 32 ： 0.9790209790209791
图片 33 ： 0.9859154929577465
图片 34 ： 0.993006993006993
图片 35 ： 0.9861111111111112
图片 36 ： 0.9861111111111112
图片 37 ： 0.9722222222222222
图

In [108]:
# 发现疑问：每次执行只能检测前92张图片的文本
# 另外执行才有显示

132张图片的正文的符合度：
图片 93 ： 0.9859154929577465
图片 94 ： 0.9861111111111112
图片 95 ： 0.993006993006993
图片 96 ： 0.9861111111111112
图片 97 ： 0.9793103448275862
图片 98 ： 0.9722222222222222
图片 99 ： 0.9861111111111112
图片 100 ： 0.9861111111111112
图片 101 ： 0.9790209790209791
图片 102 ： 0.993006993006993
图片 103 ： 0.9793103448275862
图片 104 ： 0.9790209790209791
图片 105 ： 0.993006993006993
图片 106 ： 0.9861111111111112
图片 107 ： 0.0
图片 108 ： 0.971830985915493
图片 109 ： 0.9219858156028369
图片 110 ： 0.9859154929577465
图片 111 ： 0.9659863945578231
图片 112 ： 0.9861111111111112
图片 113 ： 0.0
图片 114 ： 0.993006993006993
图片 115 ： 0.993006993006993
图片 116 ： 0.993006993006993
图片 117 ： 0.9790209790209791
图片 118 ： 0.9594594594594594
图片 119 ： 0.9655172413793104
图片 120 ： 0.9790209790209791
图片 121 ： 0.971830985915493
图片 122 ： 0.9655172413793104
图片 123 ： 0.8428571428571429
图片 124 ： 0.993006993006993
图片 125 ： 0.9790209790209791
图片 126 ： 0.9861111111111112
图片 127 ： 0.9861111111111112
图片 128 ： 0.993006993006993
图片 129 ： 0.9930069930069

发现 “图片107”和“图片113” 符合度为0
通过查看对应的文本，得到：

In [113]:
# 查看 图片107 的文本
diat

{'words_result': [{'words': '4G 111 0.9K/ss'},
  {'words': '08:53'},
  {'words': '8HD83C装'},
  {'words': '<滓精'},
  {'words': '蘑莹'},
  {'words': '宥茉着踉我起冤芗加大学生茬线羹洲'},
  {'words': '自的,4天目(辫詈娑習、冕品分菥'},
  {'words': '闷里芸汉孥)①3,剁閑课縈卧間锼自'},
  {'words': '己的泶职技罷,羋富自芑的人觜厉。'},
  {'words': '征线茗芗茄茬锼晏前苗,个发'},
  {'words': '买刀酱,:嵩着时'},
  {'words': '昨关2/:38'},
  {'words': '蘑莹'},
  {'words': '昨大2/:46'},
  {'words': '昕芗加避苜的茼学髋,鼗芩苜刘我提'},
  {'words': '宥鄴助,芠颦最茉湍就岢以加'}],
 'log_id': 1303345414308626432,
 'words_result_num': 16}

发现图片乱码，猜测是该手机系统文字问题

In [115]:
# 查看 图片113 的文本
diat

{'words_result': [{'words': '上年0:1014K区显'},
  {'words': '90'},
  {'words': '洋情'},
  {'words': 'T3DL2M'},
  {'words': '有有跟我一起免费参太学生在线实训'},
  {'words': '项⑨的,4太项(抖音运营、竞品分杯、阿'},
  {'words': '狸园认证等)3,利用课余时间锻自己的'},
  {'words': '求聆技能,丰富自卫的个人简历'},
  {'words': '在线免费报名参加在线实训项丰富个筒'},
  {'words': '实习历,9et薪oer'},
  {'words': '昨天2048刪除'},
  {'words': 'T3DL2M'},
  {'words': '昨天2048'},
  {'words': '所参加过项回的同学说,这个项对我们挺'},
  {'words': '有帮助,文章最未端就可以报名参加'},
  {'words': '评论'},
  {'words': 'e发'}],
 'log_id': 1303346417338679296,
 'words_result_num': 17}

发现正文与标准文本差距较大。

故这两个都需要人工处理

In [124]:
# 附录

# 正常图片全内容文本
contont = diat['words_result']
contont

[{'words': '中国移动图●'},
 {'words': '中国移动'},
 {'words': '2m>4@09:50'},
 {'words': '详情'},
 {'words': '·有木有跟我一起免费参加大学生在线'},
 {'words': '实训项目的,4大项目(抖音运营'},
 {'words': '竞品分析、阿里云认证等)①3,利用'},
 {'words': '课余时间锻炼自己的求职技能,丰富'},
 {'words': '自己的个人简历。'},
 {'words': '在线免费报名参加在线实训项目,丰'},
 {'words': '实习富个人简历,ge高薪ofer'},
 {'words': '昨天20:52删除'},
 {'words': '昨天20:52'},
 {'words': '听参加过项目的同学说,这个项目对'},
 {'words': '我们挺有帮助,文章最末端就可以报'},
 {'words': '名参加。'}]

#### 转发图片包含

In [2]:
# 把目标图片放入原图片 算法

# 需要输入原图片路径、目标图片路径、阈值（暂时没弄清楚有什么用）
def mathc_img(image,Target,value):

    #加载原始RGB图像
    img_rgb = cv2.imread(image)
    
    #创建一个原始图像的灰度版本，所有操作在灰度版本中处理，然后在RGB图像中使用相同坐标还原
    img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
    
    #加载将要搜索的图像模板
    template = cv2.imread(Target,0)
    
    #记录图像模板的尺寸
    w, h = template.shape[::-1]
    
    # 显示尺寸
    print(w,h)
    
    # 使用matchTemplate在原始图像中查找并匹配图像模板中的内容，并设置阈值。
    #使用matchTemplate对原始灰度图像和图像模板进行匹配
    res = cv2.matchTemplate(img_gray,template,cv2.TM_CCOEFF_NORMED)
    
    #设定阈值
    threshold = value
    
    #res大于70%
    loc = np.where( res >= threshold)
    
    # 显示方位？
    print(loc)
    
    # 匹配完成后在原始图像中使用灰度图像的坐标对原始图像进行标记。
    #使用灰度图像中的坐标对原始RGB图像进行标记
    for pt in zip(*loc[::-1]):
        cv2.rectangle(img_rgb, pt, (pt[0] + w, pt[1] + h), (7,249,151), 2) 
        # cv2.rectangle(image, 左下角坐标, 右上角坐标, color, 线条粗度)
    
    #显示图像   
    cv2.imshow('Detected',img_rgb)
    
    # 停留方法
    cv2.waitKey(0)
    
    # 关闭窗口方法
    cv2.destroyAllWindows()

In [3]:
# 导入切割作业时的json，便于寻找对应图片

import json

# 用utf8打开文件
with open("202009041513_export_json.json","r",encoding='utf8') as f:
    
    # 输出json文件
    data = json.load(f)

In [6]:
# 图片数
photo = 0

# 从上述josn文件里遍历，每到一个图片处理一次
for i in data:
    try:
        
        # 图片计数
        photo +=1
        
        # 获取图片名
        a = i['imageName']
        
        # 验证图片名
        print(a)
        
        # 通过图片名找到对应的图片
        image=("./ori/%s"%a)

        # 通过图片计数找到对应的目标图片
        # 通过观察发现目标图片处于第九个或第十个位置
        Target1=('./output/%d_9.jpg'%photo)
        Target2=('./output/%d_10.jpg'%photo)
        
        # 设定阈值
        value=0.9
        
        # 执行算法，对每张图片执行两次，找出期望的被标记后的图片
        mathc_img(image,Target1,value)
        mathc_img(image,Target2,value)
        
    except:
        pass

mmexport1594864868281.jpg
889 207
(array([260], dtype=int64), array([156], dtype=int64))
96 100
(array([488], dtype=int64), array([177], dtype=int64))
mmexport1594864871463.jpg
48 49
(array([219], dtype=int64), array([65], dtype=int64))
224 39
(array([224], dtype=int64), array([118], dtype=int64))
mmexport1594864875428.jpg
849 220
(array([314], dtype=int64), array([180], dtype=int64))
144 146
(array([574, 575, 575, 575, 576], dtype=int64), array([195, 194, 195, 196, 195], dtype=int64))
mmexport1594864880027.png
1020 257
(array([298], dtype=int64), array([188], dtype=int64))
123 131
(array([579, 579, 579], dtype=int64), array([195, 196, 197], dtype=int64))
mmexport1594864914143.jpg
101 99
(array([250], dtype=int64), array([159], dtype=int64))
558 41
(array([283], dtype=int64), array([277], dtype=int64))
mmexport1594864889614.jpg
144 152
(array([692, 693, 693, 693, 694], dtype=int64), array([225, 224, 225, 226, 225], dtype=int64))
620 106
(array([725], dtype=int64), array([389], dtype=in

84 80
(array([367], dtype=int64), array([120], dtype=int64))
mmexport1594865058722.jpg
87 97
(array([426], dtype=int64), array([129], dtype=int64))
536 91
(array([428], dtype=int64), array([222], dtype=int64))
mmexport1594865061044.jpg
153 140
(array([644, 645, 645, 645, 646], dtype=int64), array([189, 188, 189, 190, 189], dtype=int64))
673 156
(array([643], dtype=int64), array([357], dtype=int64))
mmexport1594864996079.jpg
203 63
(array([1040, 1210], dtype=int64), array([199, 785], dtype=int64))
94 72
(array([1034], dtype=int64), array([421], dtype=int64))


通过执行后的标记图片的显示，人工把不符合要求的去掉，并整合在“分享图片包含.docx”内，其中在50个图片内发现一个不合格图片（分享图片未正常显示）